In [ ]:
import xarray as xr
import os
import numpy as np
from datetime import datetime

file_path = "C:/Users/customer/Downloads/adaptor.mars.internal-1686634317.6059375-22265-12-cadc50c5-a662-48f1-b391-6e241067044a.nc"
ds = xr.open_dataset(file_path)

unique_latitudes = ds['latitude'].values
unique_longitudes = ds['longitude'].values
unique_pressure_levels = ds['level'].values 
unique_time = ds['time'].values.flatten()

date1 = "2011-01-05"

buoy_data = {
    'AD06': {'lat': 18.31, 'lon': 67.33},
    'AD07': {'lat': 14.92, 'lon': 68.98},
    'AD08': {'lat': 12.06, 'lon': 68.62},
    'AD09': {'lat': 8.11, 'lon': 73.28},
    'AD10': {'lat': 10.32, 'lon': 72.6},
    'BD08': {'lat': 17.83, 'lon': 89.2},
    'BD09': {'lat': 17.46, 'lon': 89.15},
    'BD10': {'lat': 16.33, 'lon': 87.99},
    'BD11': {'lat': 13.46, 'lon': 84.15},
    'BD12': {'lat': 10.46, 'lon': 94.12},
    'BD13': {'lat': 14, 'lon': 87},
    'BD14': {'lat': 6.6, 'lon': 88.4}
}

for buoy_id, coordinates in buoy_data.items():
    buoy_data[buoy_id]['lat'] = round(coordinates['lat'] * 2) / 2
    buoy_data[buoy_id]['lon'] = round(coordinates['lon'] * 2) / 2

# Filter out buoy locations from unique_latitudes and unique_longitudes
filtered_latitudes = [i['lat'] for i in buoy_data.values()]
filtered_longitudes = [i['lon'] for i in buoy_data.values()]

output_directory = f"data_{date1}"
os.makedirs(output_directory, exist_ok=True)

print("Date\t\t\tTime\t\t\tTemp(500mb)\t\t\tTemp(850mb)")
for lat,lon in zip(filtered_latitudes,filtered_longitudes):
        
        myfile = f"data_of_{lat}E_{lon}N.txt"
        output_file = os.path.join(output_directory, myfile) 
        f=open(output_file,'a')
        f.write(" Date\t\t\tTime\t\t\tAt 500mb\t\t\tAt 850mb\n")

        print(f"Data {lat}E /{lon}N on {date1} at 500mb")
        for time1 in unique_time:
            data_for_params = ds.sel(latitude=lat, longitude=lon, time=time1, method='nearest')
            time_data_array = str(np.array(time1, dtype='datetime64'))
            date, time1 = time_data_array.split("T")
            time2, unn = time1.split(".")
            time_val = time2.replace(":", "-")
            if(date==date1):
                
                temperature_data = data_for_params['t']
                pressure_data = data_for_params['level']

                pressure_at_500 = pressure_data.sel(level=500).values
                pressure_at_850 = pressure_data.sel(level=850).values

                temperature_at_500 = int(temperature_data.sel(level=500).values.round(decimals=4))
                celcius_at_500 = int(temperature_data.sel(level=500).values - 273.15)

                temperature_at_850 = int(temperature_data.sel(level=850).values.round(decimals=4))
                celcius_at_850 = int(temperature_data.sel(level=850).values - 273.15)

                with open(output_file, 'a') as f:
                    print(f"{time2} :{temperature_at_500}K or {celcius_at_500}deg C ")
                    f.write(f"{date1}\t\t{time2}\t\t{temperature_at_500}K {celcius_at_500}C\t\t{temperature_at_850}K {celcius_at_850}C\n")
                    f.close()
print("All files are written")